In [ ]:
#pip install pickle-mixin
#pip install fuzzywuzzy
#pip install pyarrow

In [1]:
import pandas as pd
import os
import pickle
import json
from fuzzywuzzy import fuzz

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Business: Contiene información del comercio, incluyendo localización, atributos y categorías.


In [7]:
# Ruta al archivo .pkl que deseas abrir
ruta_archivo_pkl = '../PROYECTO_FINAL_EQUIPO/Yelp/business.pkl'

# Abrir el archivo .pkl en modo lectura binaria
with open(ruta_archivo_pkl, 'rb') as archivo_pkl:
    # Cargar el objeto desde el archivo .pkl
    business_Yelp = pickle.load(archivo_pkl)


In [8]:
business_Yelp.shape

(150346, 28)

In [9]:
business_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

In [10]:
#Eliminando Columnas Duplicadas
business_Yelp = business_Yelp.loc[:, ~business_Yelp.columns.duplicated()]


In [11]:
business_Yelp.sample(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
79178,Lpm7-UfV8NUp0GzSIvaULw,Haddonview Apartments,1 MacArthur Blvd,Haddon Township,LA,08108,39.902584,-75.060111,3.0,5,1,None,"Home Services, Real Estate, Apartments","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
136359,u666lzKv1PoWnr2GS_hWbw,Marc’s Moving Services,7901 W White Water,Tampa,CA,33637,28.019117,-82.366249,4.5,5,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Couriers & Delivery Services, Movers, Home Ser...","{'Monday': '6:0-18:0', 'Tuesday': '6:0-18:0', ..."


Dejar solo los negocios que sean del tipo Restaurantes.

In [12]:
#Generar una lista de categorías únicas:

# Rellena los valores nulos con una cadena vacía
business_Yelp['categories'].fillna('', inplace=True)

# Dividir la columna "categories" en listas de categorías
categories_lists = business_Yelp['categories'].str.split(', ')

# Unir todas las listas en una única lista
all_categories = [category for categories_list in categories_lists for category in categories_list]

# Convertir el conjunto de nuevo en una lista si es necesario
unique_categories = set(all_categories)
unique_categories_list = list(unique_categories)

In [18]:
#Con las categorías únicas y nuestra palabra clave, obtendremos similitud de categorías
#Para asegurar seleccionar todos los restaurantes
keyword = "restaurant"

# Función para calcular la similitud entre la palabra clave y cada categoría
def calcular_similitud(categoria, keyword):
    return fuzz.ratio(keyword, categoria)

# Calcular la similitud para cada categoría en la lista
similitudes = [(categoria, calcular_similitud(categoria, keyword)) for categoria in unique_categories_list]

# Filtrar categorías que tienen una similitud mayor que un umbral (por ejemplo, 70)
umbral_similitud = 63  # Puedes ajustar este valor según tus necesidades
categorias_similares = [categoria for categoria, similitud in similitudes if similitud >= umbral_similitud]

In [19]:
categorias_similares

['Restaurants', 'Pop-Up Restaurants']

In [20]:
df_business_Yelp = business_Yelp
# Primero, dividimos los valores de 'categories' en listas de strings
df_business_Yelp['categories'] = df_business_Yelp['categories'].str.split(',')

df_business_Yelp = df_business_Yelp.dropna(subset=['categories'])

#Filtrar por categoría "Restaurantes"
df_business_Yelp = df_business_Yelp[df_business_Yelp['categories'].apply(lambda categories: 'Restaurants' in categories or 'Pop-Up Restaurants' in categories)]

# Unir los valores de la lista en categorías en un solo string separado por comas
df_business_Yelp['categories'] = df_business_Yelp['categories'].apply(lambda x: ','.join(x))

# Recetar el Indice
df_business_Yelp.reset_index(inplace=True, drop=True)

In [21]:
df_business_Yelp.shape

(15305, 14)

In [22]:
df_business_Yelp.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,AB,89502,39.476117,-119.789339,2.5,339,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, Italian","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [24]:
#Aunque no hay duplicados, debemos quedarnos con los restaurantes que tengamos reseñas.
df_business_Yelp['business_id'].duplicated().sum()

0

## DataSet Review

In [25]:
# Ruta de la carpeta de datos de Google Maps
yelp_folder = '/Users/alejandrobenjaminnunezguzman/Desktop/PROYECTO_FINAL_EQUIPO/Yelp'

# Ruta de la sub-carpeta "metadata-sitios"
yelp_subfolder = os.path.join(yelp_folder, 'Reviews')

In [26]:
reviews = []

# Recorrer todos los archivos
for archivo in os.listdir(yelp_subfolder):
        # Ruta del archivo JSON actual
        ruta_archivo = os.path.join(yelp_subfolder, archivo)

        # Leer el contenido del archivo
        with open(ruta_archivo) as f:
            contenido_json = f.readlines()

        # Cargar cada objeto JSON por separado
        for linea in contenido_json:
            try:
                data = json.loads(linea)
            except json.JSONDecodeError as e:
                print(f"Error al cargar el archivo JSON {ruta_archivo}: {e}")
                continue

            reviews.append(data)

# Convertir los resultados a un pandas DataFrame
df_reviews = pd.DataFrame(reviews)

In [27]:
df_reviews.shape

(6990280, 9)

Seleccionamos únicamente reviews de negocios del tipo: Restaurantes

In [30]:
df_business_Yelp.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
10948,3yZqLrzQiM8T00pZVZ43Rw,Yanaki Japanese Restaurant & Asian Cuisine,650 S Vista Ave,Boise,NJ,83705,43.598118,-116.213258,4.5,21,1,"{'RestaurantsDelivery': 'True', 'WiFi': 'u'no'...","Restaurants, Japanese, Korean, Asian Fusion","{'Monday': '0:0-0:0', 'Tuesday': '16:30-21:0',..."
4654,5guS25GDbxMEnsw_ZD7umA,Schlotzsky's,3270 E Valencia,Tucson,AB,85706,32.133696,-110.922383,3.0,82,1,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...","Restaurants, Pizza, Burgers, Delis, Caterers, ...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-21:0', '..."
179,SSxXyZFkA8U_DOeWzrrhCA,Starr Diner,785 Starr St,Phoenixville,AB,19460,40.123151,-75.508776,2.0,13,0,"{'Alcohol': 'u'none'', 'Caters': 'False', 'Goo...","Restaurants, Diners",None


In [28]:
# Crear una lista de IDs a partir de df1
lista_de_ids = df_business_Yelp['business_id'].tolist()

# Filtrar df2 basándose en la lista de IDs de df1
df_reviews_r = df_reviews[df_reviews['business_id'].isin(lista_de_ids)]

In [29]:
df_reviews_r.shape

(1192401, 9)

In [30]:
df_reviews_r.sample(2)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
2460731,Mwbq1fxD0hz7tcCRcBi-Xw,WlzwuGCMcykTYpTSVUHyBA,jEvoDXtF3xMJsiPPbqGSmw,5.0,3,0,0,Came here yesterday for the first time at arou...,2016-03-11 01:50:24
3496762,i5G78BiMFr1P3Ezm3RpLOQ,4YOU43D1rXFAkNuoiI-ZZA,ETocc91Bm5-EMPhODTUx_Q,5.0,1,0,1,"Absolutely love this place. Amazing food, amaz...",2018-10-04 18:20:59


### Corroborar que todos los restaurantes tienen reviews

In [35]:
df_business_Yelp.shape

(15305, 14)

In [42]:
# Obtén una lista de todos los ID de restaurantes únicos en df_business_Yelp
restaurant_ids = df_business_Yelp['business_id'].unique()

# Verifica si cada ID de restaurante está en la lista de ID de reseñas en df_reviews
are_all_restaurant_ids_in_reviews = all(df_reviews_r['business_id'].isin(restaurant_ids))

if are_all_restaurant_ids_in_reviews:
    print("Todos los ID de restaurantes se encuentran en la lista de reseñas.")
else:
    print("Al menos un ID de restaurante no se encuentra en la lista de reseñas.")

Todos los ID de restaurantes se encuentran en la lista de reseñas.


In [59]:
df_business_Yelp.to_csv('Restaurantes_Yelp.csv',index=False)

In [60]:
df_reviews_r.to_csv('Restaurantes_Yelp_Reviews.csv',index=False)

USER: Data del usuario incluyendo referencias a otros usuarios amigos y a toda la metadata asociada al usuario.

In [45]:
archivo_parquet = "../PROYECTO_FINAL_EQUIPO/Yelp/user-001.parquet"

# Abre el archivo Parquet y crea un DataFrame.

df_user = pd.read_parquet(archivo_parquet)

In [46]:
df_user.shape

(2105597, 22)

In [47]:
#Analizar duplicados:
duplicados = df_user[df_user['user_id'].duplicated()]
print(duplicados.shape)
# Ordena los registros duplicados por 'user_id'
duplicados_ordenados = duplicados.sort_values(by='user_id')

(117700, 22)


In [49]:
duplicados_ordenados.head(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
2017202,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24 13:20:46,63,4,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,...,0,0,0,0,0,2,0,0,2,0
2080502,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24 13:20:46,63,4,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,...,0,0,0,0,0,2,0,0,2,0


In [50]:
#Eliminar duplicados:
df_user = df_user.drop_duplicates(subset='user_id') 

#### Filtrar usuarios que han reseñado restaurantes

In [51]:
#Hay 1,987,897  de usuarios únicos
df_user.shape

(1987897, 22)

In [61]:
df_user.sample(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
1527957,doCBs1eY8AzG_JfPVUr0IQ,Adam,2,2017-06-03 00:15:02,0,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
1209482,VxwIPabJxoasX_N4FD8zVA,Mathew,17,2010-06-20 22:17:35,46,4,4,,"a4Z8X3ic5P1YDtNzMlxCMg, IFYkRXw4DwrXqPyLptOaAA...",0,...,1,0,0,0,0,0,0,0,0,0


In [52]:
#Ahora filtramos los usuarios únicamente que han echo alguna review
# en  los restaurantes. 
lista_de_id = df_reviews_r['user_id'].tolist()

# Filtrar basándose en la lista de IDs que aparecen en Restaurant 
df_user_review_r = df_user[df_user['user_id'].isin(lista_de_id)]

In [53]:
#Nos quedamos con 589,377 usuarios. 
df_user_review_r.shape

(589592, 22)

In [62]:
df_user_review_r.to_csv('Restaurantes_User_Yelp.csv',index=False)

### Tips (consejos) escritos por el usuario. 
Los tips son más cortas que las reseñas y tienden a dar sugerencias rápidas.


In [54]:
ruta_archivo_json = "/Users/alejandrobenjaminnunezguzman/Desktop/PROYECTO_FINAL_EQUIPO/Yelp/tip.json"

lista_objetos_json = []

# Abre el archivo JSON y lee línea por línea
with open(ruta_archivo_json, "r") as archivo_json:
    for linea in archivo_json:
        try:
            # Intenta cargar la línea como un objeto JSON
            objeto_json = json.loads(linea)
            lista_objetos_json.append(objeto_json)
        except json.JSONDecodeError as e:
            print(f"Error al cargar línea: {linea}")
            continue

# Crea un DataFrame a partir de la lista de objetos JSON
df_tip = pd.DataFrame(lista_objetos_json)

In [55]:
df_tip.shape

(908915, 5)

In [56]:
df_tip.head(3)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0


In [57]:
#Ahora filtramos los usuarios que únicamente han dejado tips
#en  los restaurantes. 
df_tip_r = df_tip[df_tip['user_id'].isin(lista_de_id)]

In [58]:
#Obtenemos 640,808 tips de diferentes usuarios
df_tip_r.shape

(640847, 5)

In [63]:
df_tip_r.to_csv('Restaurantes_Yelp_Tips.csv',index=False)